In [1]:
import pandas as pd

from pycaret.classification import *
from shared_utilities import helpers

from importlib import reload
reload(helpers)

<module 'shared_utilities.helpers' from '/Users/ryankazmerik/Git/data-sci/shared_utilities/shared_utilities/helpers.py'>

In [2]:
df = helpers.get_product_propensity_dataset(
    cluster="prod-app",
    database="stlrmls",
    lkupclientid="31",
    start_year=2010,
    end_year=2021
)

df.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(116095, 24)

In [3]:
df.head()

,lkupclientid,clientcode,dimcustomermasterid,distance,seasonyear,events_prior,attended_prior,events_last,attended_last,tenure,atp_last,product_current,product_last,spend_current,volume_current,attendance_current,events_current,sends,sends_prior,opens,opens_prior,date_last_send,date_last_touch,date_last_save
0,31,SACFC,457921207,69.34,2019,1,1,1,1,0,37.28,None,None,0.0,0,0,0,0.0,6.0,0.0,10.0,2022-06-24,None,2022-05-03
1,31,SACFC,457921207,69.34,2021,1,1,0,0,2,0.00,None,None,0.0,0,0,0,0.0,6.0,0.0,10.0,2022-06-24,None,2022-05-03
2,31,SACFC,457921209,10.34,2015,1,0,1,0,0,26.00,None,Individual,0.0,0,0,0,7.0,33.0,2.0,10.0,2022-07-15,None,2022-05-03
3,31,SACFC,457921209,10.34,2016,1,0,0,0,1,0.00,None,None,0.0,0,0,0,7.0,33.0,2.0,10.0,2022-07-15,None,2022-05-03
4,31,SACFC,457921209,10.34,2017,1,0,0,0,2,0.00,None,None,0.0,0,0,0,7.0,33.0,2.0,10.0,2022-07-15,None,2022-05-03


In [11]:
product_plans = ["Full Season", "Half Season", "Mini/Flex Plan"]

df = df[df["product_current"].isin(product_plans) | df["product_last"].isin(product_plans)]
df = df.reset_index(drop=True)

In [12]:
df['product_current'].value_counts()

Full Season       1490
None              1116
Individual          92
Mini/Flex Plan      39
Group               17
Half Season          8
Name: product_current, dtype: int64

In [13]:
df_train = df.sample(frac=0.70, random_state=786)
df_train = df_train.reset_index(drop=True)

df_eval = df.drop(df_train.index)
df_eval = df_eval.reset_index(drop=True)

print(df_train.shape)
print(df_eval.shape)

(1933, 24)
(829, 24)


In [14]:
df_train["product_current"].value_counts()

Full Season       1050
None               771
Individual          71
Mini/Flex Plan      23
Group               12
Half Season          6
Name: product_current, dtype: int64

In [15]:
df_eval["product_current"].value_counts()

Full Season       441
None              342
Individual         26
Mini/Flex Plan     11
Group               6
Half Season         3
Name: product_current, dtype: int64

In [16]:
setup(
    data=df_train,
    target="product_current",
    train_size=0.90,
    data_split_shuffle=True,
    silent=True,
    ignore_features=[
        "date_last_send",
        "date_last_save",
        "date_last_touch"
    ]
);

,Description,Value
0,session_id,8398
1,Target,product_current
2,Target Type,Multiclass
3,Label Encoded,"Full Season: 0, Group: 1, Half Season: 2, Individual: 3, Mini/Flex Plan: 4, None: 5"
4,Original Data,"(1933, 24)"
5,Missing Values,True
6,Numeric Features,15
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [17]:
best_model = create_model("rf");

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9770,0.0000,0.5979,0.9608,0.9687,0.9569,0.9572
1,0.9713,0.0000,0.5638,0.9550,0.9629,0.9456,0.9465
2,0.9770,0.0000,0.5714,0.9661,0.9712,0.9571,0.9576
3,0.9655,0.0000,0.5407,0.9487,0.9570,0.9357,0.9361
4,0.9770,0.9961,0.4762,0.9658,0.9713,0.9573,0.9575
5,0.9713,0.9986,0.4738,0.9550,0.9629,0.9462,0.9470
6,0.9713,0.9988,0.4722,0.9502,0.9605,0.9464,0.9469
7,0.9770,0.9949,0.5000,0.9608,0.9687,0.9573,0.9576
8,0.9770,0.9983,0.5556,0.9660,0.9704,0.9569,0.9575
